In [1]:
import torch
import torch.nn as nn

class SineRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, output_size=1, num_layers=1):
        super(SineRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size)


In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Generate sine wave data
def generate_sine_wave(seq_length, num_samples):
    X = np.linspace(0, num_samples * np.pi, seq_length * num_samples)
    y = np.sin(X)
    return y

seq_length = 50
num_samples = 100
sine_wave = generate_sine_wave(seq_length, num_samples)

# Prepare data for RNN
X = []
y = []
for i in range(len(sine_wave) - seq_length):
    X.append(sine_wave[i:i+seq_length])
    y.append(sine_wave[i+seq_length])

X = np.array(X)
y = np.array(y)

X = torch.from_numpy(X).float().unsqueeze(-1)
y = torch.from_numpy(y).float().unsqueeze(-1)



In [3]:
# Training parameters
batch_size = 10
num_epochs = 200
learning_rate = 0.001

# Initialize the model, loss function and optimizer
model = SineRNN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# Training loop
for epoch in range(num_epochs):
    hidden = model.init_hidden(batch_size)
    for i in range(0, len(X) - batch_size, batch_size):
        inputs = X[i:i+batch_size]
        targets = y[i:i+batch_size]
        
        # Detach hidden state to prevent backpropagating through the entire history
        hidden = hidden.detach()
        
        # Forward pass
        outputs, hidden = model(inputs, hidden)
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



Epoch [10/200], Loss: 0.0004
Epoch [20/200], Loss: 0.0029
Epoch [30/200], Loss: 0.0009
Epoch [40/200], Loss: 0.0000
Epoch [50/200], Loss: 0.0000
Epoch [60/200], Loss: 0.0000
Epoch [70/200], Loss: 0.0003
Epoch [80/200], Loss: 0.0000
Epoch [90/200], Loss: 0.0000
Epoch [100/200], Loss: 0.0000
Epoch [110/200], Loss: 0.0005


KeyboardInterrupt: 

In [4]:
# Generate sine wave data
model.eval()
predictions = []
input_seq = X[:1]

with torch.no_grad():
    hidden = model.init_hidden(1)
    for _ in range(100):  # Generate 100 time steps
        output, hidden = model(input_seq, hidden)
        predictions.append(output.item())
        input_seq = torch.cat((input_seq[:, 1:, :], output.unsqueeze(0).unsqueeze(0)), dim=1)

# Plot the generated sine wave
plt.plot(predictions, label='Generated')
plt.plot(sine_wave[:len(predictions)], label='Original')
plt.legend()
plt.show()


RuntimeError: Tensors must have same number of dimensions: got 3 and 4